In [1]:
import numpy as np
import quaternion
import math
from numpy import linalg as LA
from sympy.solvers import *
from sympy import symbols,cos,sin,sqrt,acos,simplify
from sympy.algebras.quaternion import Quaternion

### EXPRESSION OF NEW FRAME ###

x0 = [1, 0, 0]
y0 = [0, 1, 0]
z0 = [0, 0, 1]

x0_quat = Quaternion(0,1,0,0)
y0_quat = Quaternion(0,0,1,0)
z0_quat = Quaternion(0,0,0,1)

beta,a,b,c = symbols('beta a b c')


### GOAL VECTOR (the desired Z axis)###
goal = [a,b,c]


goal_norm = []
for i in goal:
    goal_norm.append(i/sqrt(goal[0]**2+goal[1]**2+goal[2]**2)) #Normalized vector of goal
    
### VECTOR AND ANGLE OF ROTATION ###
vector= np.cross(z0,goal_norm)


vector_norm = [] #Normalized vector of rotation
for i in vector:
    vector_norm.append(i/sqrt(vector[0]**2+vector[1]**2+vector[2]**2))
print("vector = ",vector_norm)
alpha = acos(np.vdot(z0,goal_norm)) #Angle of rotation
print("alpha = ",alpha)

if alpha == 0 :
    v = [0.0,0.0,1.0]
else:
    v = [vector_norm[0],vector_norm[1],vector_norm[2]]

### QUATERNION OF ROTATION ###
w1 = cos(alpha/2.0)
x1 = sin(alpha/2.0)*v[0]
y1 = sin(alpha/2.0)*v[1]
z1 = sin(alpha/2.0)*v[2]

q = Quaternion(w1,x1,y1,z1) #1st rotation quaternion
q_inv = Quaternion(w1,-x1,-y1,-z1)

z_prime = q.mul(z0_quat.mul(q_inv))

w2 = cos(beta/2.0)
x2 = sin(beta/2.0)*z_prime.b
y2 = sin(beta/2.0)*z_prime.c
z2 = sin(beta/2.0)*z_prime.d

q2 = Quaternion(w2,x2,y2,z2) #Quaternion of the rotation on new z axis
q2_inv = Quaternion(w2,-x2,-y2,-z2)

new_z = q2.mul(z_prime.mul(q2_inv)) #Final Z
new_x = q2.mul((q.mul(x0_quat.mul(q_inv))).mul(q2_inv)) #Final X
new_y = q2.mul((q.mul(y0_quat.mul(q_inv))).mul(q2_inv)) #Final Y

X = [simplify(new_x.b), simplify(new_x.c), simplify(new_x.d)]
Y = [new_y.b, new_y.c, new_y.d]
Z = [new_z.b, new_z.c, new_z.d]

print("X = ",X)

/Users/augustin/.virtualenvs/spherical-joint/lib/python3.6/site-packages/quaternion/numba_wrapper.py:20: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


vector =  [-b/(sqrt(a**2/(a**2 + b**2 + c**2) + b**2/(a**2 + b**2 + c**2))*sqrt(a**2 + b**2 + c**2)), a/(sqrt(a**2/(a**2 + b**2 + c**2) + b**2/(a**2 + b**2 + c**2))*sqrt(a**2 + b**2 + c**2)), 0]
alpha =  acos(c/sqrt(a**2 + b**2 + c**2))
X =  [(-a**2*sin(0.5*beta)**2*sin(0.5*acos(c/sqrt(a**2 + b**2 + c**2)))**6 + 3*a**2*sin(0.5*beta)**2*sin(0.5*acos(c/sqrt(a**2 + b**2 + c**2)))**4 + a**2*sin(0.5*beta)**2*sin(0.5*acos(c/sqrt(a**2 + b**2 + c**2)))**2 - a**2*sin(0.5*beta)**2*cos(0.5*acos(c/sqrt(a**2 + b**2 + c**2)))**6 - a**2*sin(0.5*beta)**2 - 2*a**2*sin(0.5*acos(c/sqrt(a**2 + b**2 + c**2)))**2 + a**2 + a*b*sin(1.0*beta)*sin(1.0*acos(c/sqrt(a**2 + b**2 + c**2)))**2/2 + a*b*sin(1.0*beta)*cos(1.0*acos(c/sqrt(a**2 + b**2 + c**2))) + a*b*sin(1.0*beta)*cos(2.0*acos(c/sqrt(a**2 + b**2 + c**2)))/4 - 5*a*b*sin(1.0*beta)/4 - b**2*sin(0.5*beta)**2*sin(0.5*acos(c/sqrt(a**2 + b**2 + c**2)))**6 + 3*b**2*sin(0.5*beta)**2*sin(0.5*acos(c/sqrt(a**2 + b**2 + c**2)))**4 - 3*b**2*sin(0.5*beta)**2*sin(0.5*aco

In [7]:
### EQUATION SOLVER ###

pcx,pcy,pcz = symbols('pcx pcy pcz')
cx,cy,cz = symbols('cx cy cz')
R,Rp = symbols('R Rp')#Radius of the distal and proximal circle



Pc = [0,0,pcz]   #Coordinates of the plateform's center
C = [0,0,cz]   #Coordinates of the center of the proximal circle

q3,q1 = symbols('q3 q1')
X1,X2,X3,Y1,Y2,Y3,Z1,Z2,Z3 = symbols('X1 X2 X3 Y1 Y2 Y3 Z1 Z2 Z3')

X_ = [X1,X2,X3]
Y_ = [Y1,Y2,Y3]
Z_ = [Z1,Z2,Z3]


eq = []

for i in range(3):
    eq.append(Pc[i]+R*cos(q3)*Z_[i]+R*sin(q3)*X_[i]-(C[i]+Rp*cos(q1)*x0[i]+Rp*sin(q1)*y0[i])) 
    simplify(eq[i])
print("eq : ",eq)  #Equation to solve

A = solve(eq[2],q3)   #Solving the 3rd equation gives the q31 solution
A_list = []
for i in A:
    A_list.append(i)
print("q3 : ",A_list)



for i in range(2):
    eq[i]=(Pc[i]+R*cos(A_deg[0])*Z_[i]+R*sin(A_deg[0])*X_[i]-(C[i]+Rp*cos(q1)*x0[i]+Rp*sin(q1)*y0[i])) 
    simplify(eq[i])

res = solve([eq[0],eq[1]],q1)
res_list1=[]
for i in range(len(res)):
    res_list1.append([])
    for j in res[i]:
        res_list1[i].append(j)   #Converts the results in a list
print("q1 : ",res_list1)

for i in range(2):
    eq[i]=(Pc[i]+R*cos(A_deg[1])*Z_[i]+R*sin(A_deg[1])*X_[i]-(C[i]+Rp*cos(q1)*x0[i]+Rp*sin(q1)*y0[i])) 
    simplify(eq[i])

res = solve([eq[0],eq[1]],q1)
res_list2=[]
for i in range(len(res)):
    res_list2.append([])
    for j in res[i]:
        res_list2[i].append(j)   #Converts the results in a list
print("q1 : ",res_list2)



eq :  [R*X1*sin(q3) + R*Z1*cos(q3) - Rp*cos(q1), R*X2*sin(q3) + R*Z2*cos(q3) - Rp*sin(q1), R*X3*sin(q3) + R*Z3*cos(q3) - cz + pcz]
q3 :  [2*atan((R*X3 - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2))/(R*Z3 + cz - pcz)), 2*atan((R*X3 + sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2))/(R*Z3 + cz - pcz))]
q1 :  [[-acos(R*(X1*sin(2*atan(R*X3/(R*Z3 + cz - pcz) - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))) + Z1*cos(2*atan(R*X3/(R*Z3 + cz - pcz) - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))))/Rp) + 2*pi], [acos(R*(X1*sin(2*atan(R*X3/(R*Z3 + cz - pcz) - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))) + Z1*cos(2*atan(R*X3/(R*Z3 + cz - pcz) - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))))/Rp)]]
q1 :  [[-acos(R*(X1*sin(2*atan(R*X3/(R*Z3 + cz - pcz) + sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))) + Z1*cos(2*atan(R*X3/

In [ ]:
for i in range(len(A_deg)):
    if 0 <= A_deg[i] <= 180: #Keep the solution between 0° and 180°
        q3[I] = A_deg[i]

ecart = 1000
for i in range(len(res_deg)):
    diff = abs(res_deg[i][0]-q3[I])   #Choose the q1i solution associated to the q3i solution
    if diff < ecart:
        ecart = diff
        indice = i
q1[I] = res_deg[indice][1]
    
print("TOP : q11 = ",q1[0])
print("BOTTOM : q12 = ",q1[1])
print("MIDDLE : q13 = ",q1[2])
